In [22]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc


In [2]:
fake = faker.Faker("ru_RU")

In [15]:
from dkvs.bptree import BPTree
BPTree.HALF_INODE_SIZE = 32
BPTree.MAX_INODE_SIZE = BPTree.HALF_INODE_SIZE * 2
BPTree.HALF_LEAF_SIZE = 32
BPTree.MAX_LEAF_SIZE = BPTree.HALF_LEAF_SIZE * 2

In [16]:
class MyKey(msgspec.Struct, frozen=True, order=True):
    full_name: str

class Data(msgspec.Struct):
    data: Any

In [24]:
index = BPTree[MyKey, Data]()

In [25]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")

In [27]:
data = []
gc.collect()

736

In [28]:
%%time
for i in tqdm(range(10_000_000)):
    val = fake.name() + " - " + str(i)
    if index.insert(MyKey(val)) is None:
        print(f"Duplicate key: {val}")
    data.append(val)

  0%|          | 0/10000000 [00:00<?, ?it/s]

CPU times: user 8min 22s, sys: 6.11 s, total: 8min 29s
Wall time: 8min 26s


In [29]:
index.validate(), gc.collect()

({}, 18)

In [30]:
len(data), len(data) / (60 * 8 + 24)

(10000000, 19841.26984126984)

In [36]:
index.print_node(index.tree[index.root_node])


node-idx:179756, prev:None, next:None, level: 4  <============ *ROOT*
            |-> MyKey(full_name='Майя Робертовна Миронова - 474'): 3224, 179755


In [37]:
index.print_node(index.tree[3224])


node-idx:3224, prev:None, next:179755, level: 3 
         |-> MyKey(full_name='Агата Кузьминична Фокина - 28887'): 68, 164615
         |-> MyKey(full_name='Азарий Архипович Сазонов - 53200'): 164615, 86428
         |-> MyKey(full_name='Александр Богданович Тимофеев - 36711'): 86428, 183869
         |-> MyKey(full_name='Алина Натановна Киселева - 1283'): 183869, 46041
         |-> MyKey(full_name='Анастасия Харитоновна Нестерова - 5122'): 46041, 187738
         |-> MyKey(full_name='Анжела Ивановна Красильникова - 12110'): 187738, 99426
         |-> MyKey(full_name='Анисимова Майя Михайловна - 62705'): 99426, 198609
         |-> MyKey(full_name='Антонов Януарий Филимонович - 1145'): 198609, 24260
         |-> MyKey(full_name='Архип Феодосьевич Евсеев - 5994'): 24260, 183515
         |-> MyKey(full_name='Баранов Порфирий Тимурович - 9540'): 183515, 92506
         |-> MyKey(full_name='Беляева Фаина Степановна - 141362'): 92506, 179754
         |-> MyKey(full_name='Бобров Аристарх Ануфриев

In [38]:
index.print_node(index.tree[179755])


node-idx:179755, prev:3224, next:None, level: 3 
         |-> MyKey(full_name='Майя Робертовна Миронова - 474'): 103264, 3223
         |-> MyKey(full_name='Марина Вениаминовна Самойлова - 4393'): 3223, 133937
         |-> MyKey(full_name='Марфа Максимовна Назарова - 38618'): 133937, 70929
         |-> MyKey(full_name='Мечислав Зиновьевич Кондратьев - 12594'): 70929, 139938
         |-> MyKey(full_name='Михаил Богданович Филатов - 3164'): 139938, 31764
         |-> MyKey(full_name='Моисеев Аникита Власович - 174545'): 31764, 214937
         |-> MyKey(full_name='Морозова Надежда Владиславовна - 14467'): 214937, 115772
         |-> MyKey(full_name='Наина Кирилловна Копылова - 1642'): 115772, 60712
         |-> MyKey(full_name='Нестерова Надежда Даниловна - 14881'): 60712, 131464
         |-> MyKey(full_name='Никонов Трофим Фомич - 3809'): 131464, 15080
         |-> MyKey(full_name='Нинель Эдуардовна Фомичева - 60762'): 15080, 219385
         |-> MyKey(full_name='Носов Савватий Юльевич - 

In [11]:
with open("data.dat", "wb") as f:
    pickle.dump(data, f)
    f.flush()

In [12]:
index.print_node(index.tree[index.root_node])


node-idx:132, prev:None, next:None, level: 2  <============ *ROOT*
      |-> MyKey(full_name='Дроздов Аникита Марсович'): 1, 255
      |-> MyKey(full_name='Калинин Илья Викторович'): 255, 503
      |-> MyKey(full_name='Любосмысл Елизарович Фокин'): 503, 131
      |-> MyKey(full_name='Сафонов Пантелеймон Владиславович'): 131, 267


In [13]:
index.print_node(index.tree[255])


node-idx:255, prev:1, next:503, level: 1 
   |-> MyKey(full_name='Дроздов Аникита Марсович'): 121, 245
   |-> MyKey(full_name='Дьячков Денис Георгиевич'): 245, 442
   |-> MyKey(full_name='Евгений Дорофеевич Киселев'): 442, 33
   |-> MyKey(full_name='Евгения Евгеньевна Андреева'): 33, 471
   |-> MyKey(full_name='Евгения Руслановна Сергеева'): 471, 239
   |-> MyKey(full_name='Евдоким Богданович Ильин'): 239, 464
   |-> MyKey(full_name='Евдокимова Елена Ефимовна'): 464, 130
   |-> MyKey(full_name='Евдокия Дмитриевна Денисова'): 130, 489
   |-> MyKey(full_name='Евдокия Сергеевна Федосеева'): 489, 257
   |-> MyKey(full_name='Евпраксия Антоновна Шестакова'): 257, 497
   |-> MyKey(full_name='Евпраксия Михайловна Маркова'): 497, 63
   |-> MyKey(full_name='Евсеев Евстигней Гордеевич'): 63, 423
   |-> MyKey(full_name='Евсеева Тамара Сергеевна'): 423, 222
   |-> MyKey(full_name='Евстафий Демьянович Рожков'): 222, 452
   |-> MyKey(full_name='Евфросиния Вячеславовна Петухова'): 452, 118
   |-> MyK

In [14]:
index.print_node(index.tree[503])


node-idx:503, prev:255, next:131, level: 1 
   |-> MyKey(full_name='Калинин Илья Викторович'): 323, 186
   |-> MyKey(full_name='Каллистрат Еремеевич Лихачев'): 186, 418
   |-> MyKey(full_name='Капустин Юлиан Исидорович'): 418, 25
   |-> MyKey(full_name='Карп Гордеевич Бобров'): 25, 382
   |-> MyKey(full_name='Карпова Милица Вячеславовна'): 382, 159
   |-> MyKey(full_name='Кир Ермолаевич Нестеров'): 159, 336
   |-> MyKey(full_name='Кира Ниловна Логинова'): 336, 85
   |-> MyKey(full_name='Кириллов Роман Гурьевич'): 85, 348
   |-> MyKey(full_name='Киселев Спартак Захарьевич'): 348, 170
   |-> MyKey(full_name='Клавдия Вениаминовна Вишнякова'): 170, 311
   |-> MyKey(full_name='Клавдия Тимуровна Кузьмина'): 311, 46
   |-> MyKey(full_name='Князев Фортунат Евсеевич'): 46, 331
   |-> MyKey(full_name='Ковалева Фаина Вадимовна'): 331, 189
   |-> MyKey(full_name='Колесников Автоном Васильевич'): 189, 354
   |-> MyKey(full_name='Колобов Любим Арсеньевич'): 354, 81
   |-> MyKey(full_name='Комаров Ф